In [1]:
import pandas as pd

In [2]:
fem_act = pd.read_csv("fem_act.csv")
fem_temp = pd.read_csv("fem_temp.csv")
male_act = pd.read_csv("male_act.csv")
male_temp = pd.read_csv("male_temp.csv")


In [6]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Load data (each has shape: 20160 rows × 13 mice)
fem_act = pd.read_csv("fem_act.csv")
fem_temp = pd.read_csv("fem_temp.csv")
male_act = pd.read_csv("male_act.csv")
male_temp = pd.read_csv("male_temp.csv")

# Constants
minutes_per_day = 1440
total_minutes = fem_temp.shape[0]

# # Estrus occurs on days 2, 6, 10, 14 (0-based day index: 1, 5, 9, 13)
# estrus_days = [1, 5, 9, 13]
# estrus_minutes = []
# for day in estrus_days:
#     start = day * minutes_per_day
#     end = start + minutes_per_day
#     estrus_minutes.extend(range(start, end))

# # Filter out estrus minutes from female data
# fem_temp_no_estrus = fem_temp.copy()
# fem_temp_no_estrus.iloc[estrus_minutes] = np.nan

# Compute per-minute averages
male_avg_temp = male_temp.mean(axis=1)
female_avg_temp = fem_temp.mean(axis=1)

# Create x-axis as time in minutes
time = np.arange(total_minutes)

# Plotly line plot
fig = go.Figure()

# Add alternating shaded backgrounds to show dark/light cycles (12h:12h = 720 minutes per phase)
for start in range(0, total_minutes, 720):
    fig.add_shape(
        type="rect",
        x0=start,
        x1=start + 720,
        y0=male_avg_temp.min(),
        y1=male_avg_temp.max(),
        fillcolor="rgba(0,0,0,0.05)" if (start // 720) % 2 == 0 else "rgba(255,255,255,0)",
        layer="below",
        line_width=0,
    )

fig.add_trace(go.Scatter(
    x=time,
    y=male_avg_temp,
    mode='lines',
    name='Male',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=time,
    y=female_avg_temp,
    mode='lines',
    name='Female',
    line=dict(color='hotpink')
))

fig.update_layout(
    title="Average Core Body Temperature in Mice",
    xaxis_title="Time (minutes from lights off, t=0)",
    yaxis_title="Temperature (°C)",
    legend_title="Sex",
    template="simple_white"
)



fig.show()

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Load your data
fem_act = pd.read_csv("fem_act.csv")
fem_temp = pd.read_csv("fem_temp.csv")
male_act = pd.read_csv("male_act.csv")
male_temp = pd.read_csv("male_temp.csv")

# Constants
minutes_per_day = 1440
total_minutes = fem_temp.shape[0]

# Define estrus days: Days 2, 6, 10, 14 (0-indexed: 1, 5, 9, 13)
# estrus_days = [1, 5, 9, 13]
# estrus_minutes = []
# for day in estrus_days:
#     start = day * minutes_per_day
#     end = start + minutes_per_day
#     estrus_minutes.extend(range(start, end))

# # Filter estrus out of female data
# fem_temp_no_estrus = fem_temp.copy()
# fem_temp_no_estrus.iloc[estrus_minutes] = np.nan

# Average across individuals
male_avg_temp = male_temp.mean(axis=1)
female_avg_temp = fem_temp.mean(axis=1)

# Prepare figure
fig = go.Figure()

# Plot dark/light segments with different colors
for start in range(0, total_minutes, 720):
    end = start + 720
    time_segment = np.arange(start, end)

    # Determine if it's a dark phase
    is_dark = (start // 720) % 2 == 0

    # Add male line
    fig.add_trace(go.Scatter(
        x=time_segment,
        y=male_avg_temp.iloc[start:end],
        mode='lines',
        name='Male (Dark)' if is_dark else 'Male (Light)',
        line=dict(color='darkblue' if is_dark else 'lightskyblue'),
        showlegend=start == 0 or start == 720  # show each legend only once
    ))

    # Add female line
    fig.add_trace(go.Scatter(
        x=time_segment,
        y=female_avg_temp.iloc[start:end],
        mode='lines',
        name='Female (Dark)' if is_dark else 'Female (Light)',
        line=dict(color='deeppink' if is_dark else 'lightpink'),
        showlegend=start == 0 or start == 720
    ))

# Add alternating shaded background
for start in range(0, total_minutes, 720):
    fig.add_shape(
        type="rect",
        x0=start,
        x1=start + 720,
        y0=min(male_avg_temp.min(), female_avg_temp.min()),
        y1=max(male_avg_temp.max(), female_avg_temp.max()),
        fillcolor="rgba(0,0,0,0.05)" if (start // 720) % 2 == 0 else "rgba(255,255,255,0)",
        layer="below",
        line_width=0,
    )

# Add annotations for light/dark
for day in range(14):
    fig.add_annotation(
        x=day * 1440 + 360,
        y=male_avg_temp.max(),
        text="Dark",
        showarrow=False,
        font=dict(size=10),
        opacity=0.3
    )
    fig.add_annotation(
        x=day * 1440 + 1080,
        y=male_avg_temp.max(),
        text="Light",
        showarrow=False,
        font=dict(size=10),
        opacity=0.3
    )

fig.update_layout(
    title="Average Core Body Temperature in Mice (Outside Estrus)",
    xaxis_title="Time (minutes from lights off, t=0)",
    yaxis_title="Temperature (°C)",
    legend_title="Phase and Sex",
    template="simple_white"
)

fig.show()



In [11]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load data (each has shape: 20160 rows × 13 mice)
fem_act = pd.read_csv("fem_act.csv")
fem_temp = pd.read_csv("fem_temp.csv")
male_act = pd.read_csv("male_act.csv")
male_temp = pd.read_csv("male_temp.csv")

# Constants
total_minutes = fem_temp.shape[0]
time = np.arange(total_minutes)

# Split the data into day and night
day_time_start = 720   # 720 minutes = 12 hours after lights on
night_time_end = 720   # 720 minutes = 12 hours before lights off

# Compute averages for daytime and nighttime
male_avg_day = male_temp.iloc[day_time_start:, :].mean(axis=1)
female_avg_day = fem_temp.iloc[day_time_start:, :].mean(axis=1)
male_avg_night = male_temp.iloc[:night_time_end, :].mean(axis=1)
female_avg_night = fem_temp.iloc[:night_time_end, :].mean(axis=1)

# Create subplots (2 side-by-side)
fig = make_subplots(rows=1, cols=2, subplot_titles=('Daytime Temperature', 'Nighttime Temperature'))

# Daytime plot (first subplot)
fig.add_trace(go.Scatter(
    x=time[day_time_start:],
    y=male_avg_day,
    mode='lines',
    name='Male (Day)',
    line=dict(color='blue')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=time[day_time_start:],
    y=female_avg_day,
    mode='lines',
    name='Female (Day)',
    line=dict(color='hotpink')
), row=1, col=1)

# Nighttime plot (second subplot)
fig.add_trace(go.Scatter(
    x=time[:night_time_end],
    y=male_avg_night,
    mode='lines',
    name='Male (Night)',
    line=dict(color='blue')
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=time[:night_time_end],
    y=female_avg_night,
    mode='lines',
    name='Female (Night)',
    line=dict(color='hotpink')
), row=1, col=2)

# Update layout and titles
fig.update_layout(
    title="Core Body Temperature in Mice: Day vs. Night",
    xaxis_title="Time (minutes from lights off, t=0)",
    yaxis_title="Temperature (°C)",
    legend_title="Sex",
    template="simple_white",
    showlegend=True
)

fig.show()



In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load data (each has shape: 20160 rows × 13 mice)
fem_act = pd.read_csv("fem_act.csv")
fem_temp = pd.read_csv("fem_temp.csv")
male_act = pd.read_csv("male_act.csv")
male_temp = pd.read_csv("male_temp.csv")

# Constants
minutes_per_day = 1440
total_minutes = fem_temp.shape[0]
days = total_minutes // minutes_per_day
time = np.arange(total_minutes)

# Split the data into day and night for each day (12-hour phases)
day_time_start = 720   # Start of the day (12 hours after lights on)
night_time_end = 720   # End of the night (12 hours before lights off)

# Create empty lists to store daily average values
male_avg_day = []
female_avg_day = []
male_avg_night = []
female_avg_night = []

# Compute daily averages for males and females, separating day and night
for day in range(days):
    # Indices for the current day
    day_start = day * minutes_per_day
    day_end = day_start + minutes_per_day

    # Daytime and nighttime data for males and females
    male_day = male_act.iloc[day_start + day_time_start:day_end, :].mean(axis=0)
    female_day = fem_act.iloc[day_start + day_time_start:day_end, :].mean(axis=0)
    male_night = male_act.iloc[day_start:day_start + night_time_end, :].mean(axis=0)
    female_night = fem_act.iloc[day_start:day_start + night_time_end, :].mean(axis=0)

    # Store daily averages
    male_avg_day.append(male_day.mean())
    female_avg_day.append(female_day.mean())
    male_avg_night.append(male_night.mean())
    female_avg_night.append(female_night.mean())

# Convert lists to numpy arrays for plotting
male_avg_day = np.array(male_avg_day)
female_avg_day = np.array(female_avg_day)
male_avg_night = np.array(male_avg_night)
female_avg_night = np.array(female_avg_night)

# Create subplots (2 side-by-side)
fig = make_subplots(rows=1, cols=2, subplot_titles=('Daytime Temperature', 'Nighttime Temperature'))

# Daytime plot (first subplot)
fig.add_trace(go.Scatter(
    x=np.arange(1, days + 1),  # x-axis shows days from 1 to 14
    y=male_avg_day,
    mode='lines',
    name='Male (Day)',
    line=dict(color='blue')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=np.arange(1, days + 1),  # x-axis shows days from 1 to 14
    y=female_avg_day,
    mode='lines',
    name='Female (Day)',
    line=dict(color='hotpink')
), row=1, col=1)

# Nighttime plot (second subplot)
fig.add_trace(go.Scatter(
    x=np.arange(1, days + 1),  # x-axis shows days from 1 to 14
    y=male_avg_night,
    mode='lines',
    name='Male (Night)',
    line=dict(color='blue')
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=np.arange(1, days + 1),  # x-axis shows days from 1 to 14
    y=female_avg_night,
    mode='lines',
    name='Female (Night)',
    line=dict(color='hotpink')
), row=1, col=2)

# Update layout and titles
fig.update_layout(
    title="Core Body Temperature in Mice: Day vs. Night (14 Days)",
    xaxis_title="Day",
    yaxis_title="Average Temperature (°C)",
    legend_title="Sex",
    template="simple_white",
    showlegend=True
)

fig.show()

